In [204]:
# strongly inspired by https://towardsdatascience.com/analyzing-coronavirus-covid-19-data-using-pandas-and-plotly-2e34fe2c4edc
# download population data from https://data.worldbank.org/indicator/sp.pop.totl as csv, unzip and manually remove the first four lines and replace 'United States' with 'US' and 'Iran, Islamic Rep.' with 'Iran', save to pop.csv
## Import Libraries
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.linear_model import LinearRegression 
from sklearn.preprocessing import PolynomialFeatures 
plt.rcParams['figure.figsize'] = [15, 5]
from IPython import display
from ipywidgets import interact, widgets
import datetime

## Read Data for Cases, Deaths and Recoveries
srcconfirmed="https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
srcrecovered="https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv"
srcdeaths="https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
srcpop="pop.csv"
Cases_confirmed=pd.read_csv(srcconfirmed)
Cases_recovered=pd.read_csv(srcrecovered)
Cases_deaths=pd.read_csv(srcdeaths)
pop_raw=pd.read_csv(srcpop)

print("Data source: https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/")
print("Retrieved on: " + str(datetime.datetime.now()))

Data source: https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/
Retrieved on: 2020-04-07 19:05:28.078795


In [205]:
pop = pd.DataFrame([pop_raw['2018'].values], columns=pop_raw['Country Name'].values)
pop['Iran'].values[0]

81800269.0

In [206]:
def fix_date(df):
    return datetime.datetime.strptime(df["date"], '%m/%d/%y').strftime('%Y-%m-%d')

def fix_dataset(cases, value_name):
    cases = cases.rename(columns={'Province/State':'label', 'Country/Region':'parent'})
    cases.drop(['Lat', 'Long'], axis=1, inplace=True)
    cases = cases.melt(id_vars=['label','parent'], var_name="date", value_name=value_name)
    cases.fillna(value="", inplace=True)
    cases["date"] = cases.apply(fix_date, axis=1)
    cases.set_index(["parent","label","date"], inplace=True)
    return cases

# Transform
Cases_confirmed_clean = fix_dataset(Cases_confirmed, "confirmed")
Cases_recovered_clean = fix_dataset(Cases_recovered, "recovered")
Cases_deaths_clean = fix_dataset(Cases_deaths, "deaths")

Cases_raw = Cases_deaths_clean.merge(Cases_confirmed_clean, left_on=['parent','label','date'], right_on=['parent','label','date'])
Cases_raw = Cases_raw.merge(Cases_recovered_clean, left_on=['parent','label','date'], right_on=['parent','label','date'])
Cases_raw.loc['US']

deaths  confirmed  recovered
label date                                    
      2020-01-22       0          1          0
      2020-01-23       0          1          0
      2020-01-24       0          2          0
      2020-01-25       0          2          0
      2020-01-26       0          5          0
...                  ...        ...        ...
      2020-04-02    5926     243453       9001
      2020-04-03    7087     275586       9707
      2020-04-04    8407     308850      14652
      2020-04-05    9619     337072      17448
      2020-04-06   10783     366614      19581

[76 rows x 3 columns]

In [207]:
def add_derived(Cases):
#    Cases.set_index('date', inplace=True)
#    Cases['active'] = Cases['confirmed']-Cases['deaths']-Cases['recovered']
    Cases['lethality'] = np.round(Cases['deaths']/Cases['confirmed'], 3)
    
#    Cases.drop(['derivedConfirmed_'], axis=1, inplace=True)
    return Cases

# Get Daily Data
#Cases_diff = Cases_raw.groupby(['parent','label','date'])
#Cases_diff = Cases_diff.sum()
#Cases_diff = Cases_diff.diff().fillna(0)
#Cases_diff = Cases_diff.rename(columns={"confirmed":"confirmed_new","deaths":"deaths_new","recovered":"recovered_new"})
#Cases = Cases_raw.merge(Cases_diff, left_on=['parent','label','date'], right_on=['parent','label','date'])

Cases = Cases_raw
Cases.loc['Germany']
#Cases

deaths  confirmed  recovered
label date                                    
      2020-01-22       0          0          0
      2020-01-23       0          0          0
      2020-01-24       0          0          0
      2020-01-25       0          0          0
      2020-01-26       0          0          0
...                  ...        ...        ...
      2020-04-02    1107      84794      22440
      2020-04-03    1275      91159      24575
      2020-04-04    1444      96092      26400
      2020-04-05    1584     100123      28700
      2020-04-06    1810     103374      28700

[76 rows x 3 columns]

In [247]:
# Shift data
#CasesShiftedB = Cases.tail(0)
#for country in Cases.droplevel('date').index.unique().tolist():
#    firstcase = Cases.loc[country]['deaths'].reset_index().set_index('date')
#    firstcase = firstcase[firstcase.ne(0)].dropna().reset_index()
#    firstcase['parent'] = country
#    firstcase = firstcase.set_index(['parent', 'label', 'date'])
#    CasesShiftedB = CasesShiftedB.append(firstcase)
#CasesShiftedB

#CasesS = {}

#for place in Cases.droplevel('date').index.unique().tolist():
#    firstcase = Cases.loc[place]['deaths'].reset_index().set_index('date')
#    firstcase = firstcase[firstcase.ne(0)]
#    CasesS[place] = firstcase

#CasesS

def add_day(df):
    firstcase = Cases.loc[place]['deaths'].reset_index().set_index('date')
    Cases['day'] = Cases.loc[place][firstcase.ne(0)]
    return day

def combine_projectedDeaths(row):
    if row['deaths'] == False:
        return row['projectedDeaths']
    return row['deaths']

def add_simulated(Cases):
    Cases['deaths_bak'] = Cases['deaths']
    Cases['deaths'] = Cases.apply(combine_projectedDeaths, axis=1)
    Cases['derivedConfirmed_'] = np.round(Cases['deaths'] / 0.03, 0) # mortality
    Cases['derivedConfirmed'] = Cases['derivedConfirmed_'].shift(periods=-14) # time to death after turning infectious
    
#    Cases['derivedRecovered_'] = Cases['derivedConfirmed_'].shift(periods=-3) # periods = time to death + time to recovery
#    Cases['derivedRecovered'] = Cases['derivedRecovered_'] - Cases['deaths']
    
#    Cases['derivedActive'] = Cases['derivedConfirmed_'].shift(periods=-12) - Cases['deaths'] - Cases['derivedRecovered'] # periods = time to death + incubation period
    Cases['derivedLethality'] = np.round(Cases['deaths']/Cases['derivedConfirmed'], 3)
    Cases['testCoverage'] = np.minimum(1,np.round(Cases['confirmed']/Cases['derivedConfirmed'], 3))
    
    per1mio = ["deaths", "confirmed", "recovered", "projectedDeaths", "derivedConfirmed"]
    for col in per1mio:
        Cases[col+"_per1mio"] = np.round(Cases[col] * (1000000 / pop[place].values[0]), 0)
    
    Cases['derivedConfirmed'] = np.round(Cases['derivedConfirmed'], 0)
    Cases['deaths'] = Cases['deaths_bak']
    #Cases.drop(['derivedConfirmed_','derivedRecovered_','deaths_bak'], axis=1, inplace=True)
    Cases.drop(['derivedConfirmed_','deaths_bak'], axis=1, inplace=True)
    return Cases

#Cases_sim = add_simulated(Cases.loc['Germany'].droplevel('label'))

# parameters and dates
lookahead = 90
cutoffsimulated = -15 # days we calculate in the simulation, but later don't display
cutoffpolynomial = -14 # days we calculate in the polynomial, but later don't display
firstrecordeddate = Cases.reset_index().head(1)['date'].values[0]
firstsimdate = Cases.reset_index().head(2)['date'].values[-1]
lastrecordeddate = Cases.reset_index().tail(1)['date'].values[0]
startdate = datetime.datetime.strptime(lastrecordeddate, '%Y-%m-%d')
projectionstartdate = (startdate + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
#startdate = datetime.datetime.strptime(cases.index[-1], '%Y-%m-%d')
enddate = (startdate + datetime.timedelta(days=(lookahead + cutoffsimulated))).strftime('%Y-%m-%d')
datetoday = datetime.datetime.now().strftime('%Y-%m-%d')

def extenddata(cases, title=""):
    cases = cases.reset_index().groupby(["date"]).sum() #.reset_index().set_index('date')
    y = cases['deaths'].values

    # average every two entries
    y = y[-14:]
    #y = np.nanmean(np.pad(y.astype(float), (0, 3 - y.size%3), mode='constant', constant_values=np.NaN).reshape(-1, 3), axis=1)
    
    # calculate polynomial
    x = range(0, y.size)
    z = np.polyfit(x, y, 2)
    f = np.poly1d(z)
    print(f)

    # calculate new x's and y's
    x_new = np.linspace(x[-1], x[-1]+lookahead, lookahead)
    y_new = f(x_new)
    
    # prepare new date range
    x_new_dates = [] # in the fit graph we need to start one day early for it to look smooth
    last_val = cases['deaths'].values[-1]
    for x_delta in range(1,lookahead): # generate future dates
        date = (startdate + datetime.timedelta(days=x_delta)).strftime('%Y-%m-%d')
        x_new_dates.append(date)
        val = max(0,np.round(f(x[-1]+1+x_delta)),0)
        
        # once we are over the peak deaths, the curve ends and goes linear
        if val < last_val:
            val = last_val
        last_val = val
        
        cases = cases.append(pd.DataFrame({'date':[date], 'deaths': False, 'projectedDeaths':[val]}).set_index("date"))
#        print('delta '+date+" y "+str(f(x[-1]+1+x_delta)))

    # cut displayed data and add simulated numbers
#    date = (startdate + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
#    cases = add_simulated(cases)[date:enddate].fillna(0)
    cases = add_derived(cases)
    cases = add_simulated(cases)[firstsimdate:enddate].fillna(0)
#    y_new = y_new[:cutoffpolynomial]

    # export data
    if title != "":
        cases.to_csv("datasets/"+datetoday+"_sim_"+title+".csv")
        cases.to_html("datasets/"+datetoday+"_sim_"+title+".html")
#        cases.to_excel("output.xslx")

    # in the fit graph we need to start one day early for it to look smooth
    #cases = cases.append(pd.DataFrame({'date':[startdate], 'deaths':[f(x[-1]+1)]}).set_index("date"))
        
    return cases

GlobalTotals = Cases.reset_index().groupby('date').sum()
#GlobalTotals = extenddata(GlobalTotals)
#GlobalTotals.loc["2020-03-01":"2020-03-14"]
#GlobalTotals
#Cases.loc['US']
extenddata(Cases.loc['China'])
#extenddata(Cases.loc['Germany']).loc[lastrecordeddate]
#extenddata(Cases.loc['China'].reset_index().groupby(["label", "date"]).sum())
#extenddata(Cases.loc['Germany']).loc[projectionstartdate]
#Cases.loc['Germany']

          2
-0.03365 x + 4.708 x + 3281


,deaths,confirmed,recovered,projectedDeaths,lethality,derivedConfirmed,derivedLethality,testCoverage,deaths_per1mio,confirmed_per1mio,recovered_per1mio,projectedDeaths_per1mio,derivedConfirmed_per1mio
date,,,,,,,,,,,,,
2020-01-22,17,548.0,28.0,0.0,0.031,18767.0,0.001,0.029,2.0,54.0,3.0,0.0,1843.0
2020-01-23,18,643.0,30.0,0.0,0.028,21100.0,0.001,0.030,2.0,63.0,3.0,0.0,2072.0
2020-01-24,26,920.0,36.0,0.0,0.028,23933.0,0.001,0.038,3.0,90.0,4.0,0.0,2350.0
2020-01-25,42,1406.0,39.0,0.0,0.030,26833.0,0.002,0.052,4.0,138.0,4.0,0.0,2635.0
2020-01-26,56,2075.0,49.0,0.0,0.027,30167.0,0.002,0.069,5.0,204.0,5.0,0.0,2962.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-16,0,0.0,0.0,3446.0,0.000,114867.0,0.030,0.000,338.0,0.0,0.0,338.0,11280.0
2020-06-17,0,0.0,0.0,3446.0,0.000,114867.0,0.030,0.000,338.0,0.0,0.0,338.0,11280.0
2020-06-18,0,0.0,0.0,3446.0,0.000,114867.0,0.030,0.000,338.0,0.0,0.0,338.0,11280.0


In [256]:
%matplotlib inline
plt.style.use('seaborn-dark')

Cases = Cases.reset_index().set_index(['parent','label'])

def plotData(cases, title, show=True, sim=False):
    cases = cases.groupby('date').sum().reset_index().set_index('date')
    if type(sim) == bool and sim == False:
        sim = extenddata(cases, title)
    
#    sim = extenddata(cases[firstrecordeddate:"2020-04-04"], title)
    projected = sim[lastrecordeddate:]
    
    fig = make_subplots(rows=3, cols=2,shared_xaxes=True,
                        specs=[[{}, {}],[{},{}],
                           [{"colspan": 2}, None]],
                        subplot_titles=('Total Confirmed Cases','Active Cases','Deaths','Recoveries','Death to Cases Ratio'))
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=cases.index,y=cases['confirmed'],
                             mode='lines',
                             name='Confirmed Cases',
                             line=dict(color='red',width=2)))
#                             row=1,col=1)
    #fig.add_trace(go.Scatter(x=cases.index,y=cases['active'],
    #                         mode='lines',
    #                         name='Active Cases',
    #                         line=dict(color='purple',width=2)))
#                             row=1,col=2)
    #fig.add_trace(go.Scatter(x=cases.index,y=cases['recovered'],
    #                         mode='lines',
    #                         name='Recoveries',
    #                         line=dict(color='green',width=2)))
#                             row=1,col=1)
    fig.add_trace(go.Scatter(x=cases.index,y=cases['deaths'],
                             mode='lines',
                             name='Deaths',
                             line=dict(color='black',width=2)))
#                             row=1,col=1)
    fig.add_trace(go.Scatter(x=projected.index,y=projected['projectedDeaths'],
                             mode='lines',
                             name='Projected deaths',
                             line=dict(color='black',width=2,dash='dot')))
#                             row=1,col=1)
        
    fig.add_trace(go.Scatter(x=sim.index,y=sim['derivedConfirmed'],
                             mode='lines',
                             name='Derived Cases',
                             line=dict(color='red',width=2,dash='dot')))
                             #row=1,col=1)
    #fig.add_trace(go.Scatter(x=sim.index,y=sim['derivedActive'],
    #                         mode='lines',
    #                         name='Derived Active Cases',
    #                         line=dict(color='purple',width=2,dash='dot')))
                             #row=1,col=2)
    #fig.add_trace(go.Scatter(x=sim.index,y=sim['derivedRecovered'],
    #                         mode='lines',
    #                         name='Derived Recoveries',
    #                         line=dict(color='green',width=2,dash='dot')))
                             #row=2,col=2)

    fig.update_layout(showlegend=True)
#    fig.update_layout(title=title,
#                       yaxis_title='Cases',
#                       xaxis_title='Date')
#    fig.show()
    fig.update_layout(title=title,
#                       yaxis_title='Cases',
                       yaxis_title='Cases (log)', yaxis_type="log",
                       xaxis_title='Date')

    if show:
        fig.show()
    fig.write_html("datasets/"+datetoday+"_cases_"+title+".html")
#    fig.write_image("test.png")

#plotData(GlobalTotals, 'Global')
#plotData(Cases.loc["US"], 'US')
#plotData(Cases.loc['United Kingdom'], 'United Kingdom')
#plotData(Cases.loc['China'], 'China')
#plotData(Cases.loc['Italy'], 'Italy')
plotData(Cases.loc['Germany'], 'Germany')
#plotData(Cases.loc['Sweden'], 'Sweden')
plotData(Cases.loc['Belgium'], 'Belgium')
#plotData(Cases.loc['France'], 'France')
#plotData(Cases.loc['Spain'], 'Spain')
plotData(Cases.loc['Netherlands'], 'Netherlands')

       2
6.221 x + 47.55 x + 148.3


       2
6.141 x + 38.42 x + 116.4


      2
2.88 x + 91.78 x + 252.1


In [210]:
Cases.loc['United Kingdom']

,date,deaths,confirmed,recovered
label,,,,
Bermuda,2020-01-22,0,0,0
Cayman Islands,2020-01-22,0,0,0
Channel Islands,2020-01-22,0,0,0
Gibraltar,2020-01-22,0,0,0
Isle of Man,2020-01-22,0,0,0
...,...,...,...,...
,2020-04-06,5373,51608,135
Anguilla,2020-04-06,0,3,0
British Virgin Islands,2020-04-06,0,3,0


In [211]:
Cases.loc['Germany']

,date,deaths,confirmed,recovered
label,,,,
,2020-01-22,0,0,0
,2020-01-23,0,0,0
,2020-01-24,0,0,0
,2020-01-25,0,0,0
,2020-01-26,0,0,0
...,...,...,...,...
,2020-04-02,1107,84794,22440
,2020-04-03,1275,91159,24575
,2020-04-04,1444,96092,26400


In [249]:
countries = ["France","Italy","Spain","US","United Kingdom","Iran","China","Netherlands","Germany","Belgium","Sweden"]
sims_lastrecordeddate = {}
sims_enddate = {}
columns = {}
per1mio = ["deaths", "confirmed", "recovered", "projectedDeaths"]
for place in countries:
    print(place)
    sim = extenddata(Cases.loc[place], place)
    plotData(Cases.loc[place], place, False, sim) # export interactive plot
    
    sims_lastrecordeddate[place] = sim.loc[lastrecordeddate]
    sims_enddate[place] = sim.loc[enddate]
#    if "label" not in sims_lastrecordeddate[place].index.values:
#        sims_lastrecordeddate[place] = sims_lastrecordeddate[place].reset_index().groupby("date").sum().loc[lastrecordeddate]
#        sims_enddate[place] = sims_enddate[place].drop("label")
#    else:
#        sims_lastrecordeddate[place] = sims_lastrecordeddate[place].drop("label")
#        sims_enddate[place] = sims_enddate[place].drop("label")
    columns = sims_lastrecordeddate[place].index
    # TODO: exclude testCoverage, lethality from division
    
    #for col in per1mio:
    #    sims_lastrecordeddate[place][col] = np.round(sims_lastrecordeddate[place][col] * (1000000 / pop[place].values[0]), 0)
    #    sims_enddate[place][col] = np.round(sims_enddate[place][col]  * (1000000 / pop[place].values[0]), 0)
    print("ok")
    
pd_sims = pd.DataFrame(sims_lastrecordeddate, columns = sims_lastrecordeddate.keys())
pd_sims.insert(0, 'name', columns)
pd_sims.set_index('name', inplace=True)
pd_sims_enddate = pd.DataFrame(sims_enddate, columns = sims_enddate.keys())
pd_sims_enddate.insert(0, 'name', columns)
pd_sims_enddate.set_index('name', inplace=True)

France
       2
41.35 x + 79.62 x + 1219
ok
Italy
      2
-8.7 x + 880 x + 6647
ok
Spain
         2
-0.3537 x + 832.8 x + 2735
ok
US
       2
54.98 x + 70.02 x + 783.2
ok
United Kingdom
       2
30.18 x + 4.707 x + 434.5
ok
Iran
        2
0.1236 x + 135 x + 1952
ok
China
          2
-0.03365 x + 4.708 x + 3281
ok
Netherlands
      2
2.88 x + 91.78 x + 252.1
ok
Germany
       2
6.221 x + 47.55 x + 148.3
ok
Belgium
       2
6.141 x + 38.42 x + 116.4
ok
Sweden
       2
1.901 x + 8.985 x + 43.03
ok


In [250]:
pd_sims_enddate
#pd_sims

,France,Italy,Spain,US,United Kingdom,Iran,China,Netherlands,Germany,Belgium,Sweden
name,,,,,,,,,,,
deaths,0.000000e+00,0.00,0.000,0.000000e+00,0.000000e+00,0.000,0.00,0.000,0.000,0.000,0.000
confirmed,0.000000e+00,0.00,0.000,0.000000e+00,0.000000e+00,0.000,0.00,0.000,0.000,0.000,0.000
recovered,0.000000e+00,0.00,0.000,0.000000e+00,0.000000e+00,0.000,0.00,0.000,0.000,0.000,0.000
projectedDeaths,3.358460e+05,28899.00,74056.000,4.424740e+05,2.398980e+05,14949.000,3446.00,31231.000,53658.000,52182.000,15901.000
lethality,0.000000e+00,0.00,0.000,0.000000e+00,0.000000e+00,0.000,0.00,0.000,0.000,0.000,0.000
derivedConfirmed,1.493707e+07,963300.00,2825500.000,1.970760e+07,1.070280e+07,572400.000,114867.00,1341900.000,2368200.000,2307600.000,704567.000
derivedLethality,2.200000e-02,0.03,0.026,2.200000e-02,2.200000e-02,0.026,0.03,0.023,0.023,0.023,0.023
testCoverage,0.000000e+00,0.00,0.000,0.000000e+00,0.000000e+00,0.000,0.00,0.000,0.000,0.000,0.000
deaths_per1mio,5.014000e+03,478.00,1585.000,1.352000e+03,3.608000e+03,183.000,2.00,1812.000,647.000,4569.000,1561.000


In [251]:
totalcases = pd_sims.loc['deaths_per1mio'].sort_values(ascending=False)
top10 = totalcases.head(10)
fig = go.Figure(go.Bar(x=top10.index, y=top10.values,
                      text=top10.values,
            textposition='outside'))
fig.update_layout(title_text='Top 10 Countries by Deaths / 1 Mio on '+lastrecordeddate)
fig.update_yaxes(showticklabels=False)

fig.show()
fig.write_html("datasets/"+datetoday+"_top10_deaths.html")

In [252]:
totalcases = pd_sims.loc['testCoverage'].sort_values(ascending=False)
top10 = totalcases.head(10)
fig = go.Figure(go.Bar(x=top10.index, y=top10.values,
                      text=top10.values,
            textposition='outside'))
fig.update_layout(title_text='Top Countries by Test Coverage on '+lastrecordeddate)
fig.update_yaxes(showticklabels=False)

fig.show()
fig.write_html("datasets/"+datetoday+"_top10_testcoverage.html")

In [257]:
totalcases = pd_sims.loc['derivedConfirmed_per1mio'].sort_values(ascending=False)
top10 = totalcases.head(10)
fig = go.Figure(go.Bar(x=top10.index, y=top10.values,
                      text=top10.values,
            textposition='outside'))
fig.update_layout(title_text='Top Countries by Derived Cases / 1 Mio on '+lastrecordeddate)
fig.update_yaxes(showticklabels=False)

fig.show()
fig.write_html("datasets/"+datetoday+"_top10_derivedcases.html")

In [254]:
totalcases = pd_sims_enddate.loc['projectedDeaths_per1mio'].sort_values(ascending=False)
top10 = totalcases.head(10)
fig = go.Figure(go.Bar(x=top10.index, y=top10.values,
                      text=top10.values,
            textposition='outside'))
fig.update_layout(title_text='Top Countries by Projected Deaths  / 1 Mio on '+enddate)
fig.update_yaxes(showticklabels=False)

fig.show()
fig.write_html("datasets/"+datetoday+"_top10_projecteddeaths.html")

In [255]:
totalcases = pd_sims_enddate.loc['projectedDeaths_per1mio'].sort_values(ascending=False)
totalcases

France            5014.0
Belgium           4569.0
United Kingdom    3608.0
Netherlands       1812.0
Spain             1585.0
Sweden            1561.0
US                1352.0
Germany            647.0
Italy              478.0
Iran               183.0
China                2.0
Name: projectedDeaths_per1mio, dtype: float64